# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [12]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, InMemoryAnalogSignalSource, EventList
from ephyviewer import AnalogSignalSourceWithScatter, SpikeInterfaceRecordingSource, InMemoryEventSource

import mbTools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Choose experiment
Select the folder of the experiment to display. If the experiment was already analyzed, you can select the iterimAnalysis folder. Otherwise select the raw data recording folder.

In [13]:
theExpe = mbTools.experiment()

Local config file loaded from localConfig.ini
Local config file updated
current folder /Users/ahay/Documents/DATA/AHCR/interimAnalysis/Bleu/20240220 contains a config file
/Users/ahay/Documents/DATA/AHCR/interimAnalysis/Bleu/20240220/expeConfig1.ini
/Users/ahay/Documents/DATA/AHCR/interimAnalysis/Bleu/20240220/expeConfig1.ini saved


FileChooser(path='/Users/ahay/Documents/DATA/AHCR/interimAnalysis/Bleu/20240220', filename='', title='<b>OpenE…

## Load Data

### Map the whole data into memory

In [14]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
theExpe.analyseExpe_findData(fullSampling=False,spindleBN='SPINDproperties')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/ahay/Documents/DATA/AHCR/interimAnalysis/Bleu/channelMaps.ini
Mapping found and loaded
{'EMG': [{'canal': '20', 'status': 1}], 'Par * DS~': [{'canal': '5', 'status': 2}, {'canal': '4', 'status': 1}], 'PFC * chan 15': [{'canal': '14', 'status': 2}, {'canal': '15', 'status': 1}], 'OFC *': [{'canal': '12', 'status': 2}, {'canal': '13', 'status': 1}], 'S1 * DS+': [{'canal': '9', 'status': 2}, {'canal': '8', 'status': 1}], 'BLA': [{'canal': '10', 'status': 2}, {'canal': '11', 'status': 1}], 'mEC +': [{'canal': '2', 'status': 2}, {'canal': '3', 'status': 1}], 'CA1 +': [{'canal': '30', 'status': 1}, {'canal': '30', 'status': 0}], 'M1 * DS+': [{'canal': '16', 'status': 2}, {'canal': '17', 'status': 1}], 'RSP': [{'canal': '0', 'status': 2}, {'canal': '1', 'status': 1}], 'Perirhinal +': [{'canal': '6', 'status': 2}, {'canal': '7', 'status': 1}]}
********found some RawDataChannelExtractedDS.npy files***

In [15]:
print(theExpe.data['Spindles'])

{'M1':      peak time  Duration  peak amp  start time   end time  Max freq  Max int  \
2      56428.0     573.0       1.0     56250.0    56823.0      16.0   3453.0   
5      78944.0     885.0       1.0     78625.0    79510.0      16.0   3395.0   
6      80653.0     559.0       1.0     80293.0    80852.0      16.0   2762.0   
7      83315.0    1112.0       1.0     83112.0    84224.0      16.0   3935.0   
15    167440.0     564.0       1.0    167203.0   167767.0      16.0   3018.0   
..         ...       ...       ...         ...        ...       ...      ...   
931  9099074.0    2013.0       2.0   9097914.0  9099927.0      16.0   4271.0   
934  9109842.0    1571.0       1.0   9109610.0  9111181.0      16.0   4201.0   
937  9114500.0     386.0       1.0   9114296.0  9114682.0      16.0   3734.0   
939  9118827.0     601.0       1.0   9118408.0  9119009.0      16.0   4887.0   
940  9131291.0    1464.0       1.0   9131109.0  9132573.0      16.0   2862.0   

     Avg freq  Avg int  toKeep  

### Extract submatrix of interest

In [16]:
#initiate combined and channelLabels
combined =  {}
channelLabels = {}
sample_rates = {}
t_start = {}
#t_end = 300 #seconds

### Extract submatrix of interest

In [17]:
#LFP
if 'OE_LFP' in theExpe.data:
    sample_rates['LFP'] = theExpe.data['OE_LFP'].sampling_rate #20000
    t_start['LFP'] = theExpe.data['OE_LFP'].start
    combined['LFP'] = theExpe.data['OE_LFP'].combineStructures("All")#['M1'])
    channelLabels['LFP'] = theExpe.data['OE_LFP'].channelLabels[:]
    print("LFP data combined")
else:
    print("no LFP data to combine")

no LFP data to combine


In [18]:
theExpe.data['LFP_DS'].sampling_rate=1000
theExpe.data['LFP_DS'].start=0
print(theExpe.data['LFP_DS'].sampling_rate)

1000


In [19]:
#LFP
if 'LFP_DS' in theExpe.data:
    sample_rates['LFP_DS'] = theExpe.data['LFP_DS'].sampling_rate #20000
    t_start['LFP_DS'] = theExpe.data['LFP_DS'].start
    combined['LFP_DS'] = theExpe.data['LFP_DS'].combineStructures("All")#['M1'])
    channelLabels['LFP_DS'] = theExpe.data['LFP_DS'].channelLabels[:]
    print("LFP data combined")
else:
    print("no LFP data to combine")

dict_keys(['EMG', 'Par * DS~', 'PFC * chan 15', 'OFC *', 'S1 * DS+', 'BLA', 'mEC +', 'CA1 +', 'M1 * DS+', 'RSP', 'Perirhinal +'])
EMG -> [{'canal': '20', 'status': 1}]
Getting floating signal of channel 20 for EMG
Par * DS~ -> [{'canal': '5', 'status': 2}, {'canal': '4', 'status': 1}]
Getting differential signal of channel 5 - channel 4 for Par * DS~
PFC * chan 15 -> [{'canal': '14', 'status': 2}, {'canal': '15', 'status': 1}]
Getting differential signal of channel 14 - channel 15 for PFC * chan 15
OFC * -> [{'canal': '12', 'status': 2}, {'canal': '13', 'status': 1}]
Getting differential signal of channel 12 - channel 13 for OFC *
S1 * DS+ -> [{'canal': '9', 'status': 2}, {'canal': '8', 'status': 1}]
Getting differential signal of channel 9 - channel 8 for S1 * DS+
BLA -> [{'canal': '10', 'status': 2}, {'canal': '11', 'status': 1}]
Getting differential signal of channel 10 - channel 11 for BLA
mEC + -> [{'canal': '2', 'status': 2}, {'canal': '3', 'status': 1}]
Getting differential sign

In [20]:
print(combined['LFP_DS'])

[[   286.72464291    465.47007197   -364.34381365 ...   -321.479472
     208.13984933   -304.19478557]
 [   360.02647432    605.17853095   -494.72286115 ...   -476.81927054
     322.11389408   -424.33979195]
 [   288.9930758     540.6598268    -402.08227197 ...   -478.36132137
     364.12938751   -257.28041934]
 ...
 [  1144.13012125    691.80415105   -708.82905165 ...   -575.49523044
     125.28480116  -1445.93494408]
 [  1736.34253229    816.74627692   -764.99692497 ...   -573.27219231
      94.41668477  -1595.24030014]
 [    50.56811192    264.41274592 -16736.24226062 ...   -854.48517703
     114.86325222   -274.99371362]]


In [21]:
#NPX
if 'NPX' in theExpe.data:
    sample_rates['NPX'] = theExpe.data['NPX'].sampling_rate #30000
    t_start['NPX'] = theExpe.data['NPX'].start
    combined['NPX'] = theExpe.data['NPX'].signal['spike']
    channelLabels['NPX'] = theExpe.data['NPX'].channelLabels
    print("NPX data combined")
else:
    print("no NPX data to combine")

no NPX data to combine


In [22]:
if 'Spindles' in theExpe.data:
    structure = 'M1'
    All_Spindle = theExpe.data['Spindles'][structure]
    print(All_Spindle)

     peak time  Duration  peak amp  start time   end time  Max freq  Max int  \
2      56428.0     573.0       1.0     56250.0    56823.0      16.0   3453.0   
5      78944.0     885.0       1.0     78625.0    79510.0      16.0   3395.0   
6      80653.0     559.0       1.0     80293.0    80852.0      16.0   2762.0   
7      83315.0    1112.0       1.0     83112.0    84224.0      16.0   3935.0   
15    167440.0     564.0       1.0    167203.0   167767.0      16.0   3018.0   
..         ...       ...       ...         ...        ...       ...      ...   
931  9099074.0    2013.0       2.0   9097914.0  9099927.0      16.0   4271.0   
934  9109842.0    1571.0       1.0   9109610.0  9111181.0      16.0   4201.0   
937  9114500.0     386.0       1.0   9114296.0  9114682.0      16.0   3734.0   
939  9118827.0     601.0       1.0   9118408.0  9119009.0      16.0   4887.0   
940  9131291.0    1464.0       1.0   9131109.0  9132573.0      16.0   2862.0   

     Avg freq  Avg int  toKeep  
2     

In [23]:
#this cell can be used to plot very precisely time of interest. Beware that it conflicts with ephyviewer however. It might be possible to have 2 notebooks open simultanéeously...
if False:
    %matplotlib widget
    #you can confiure a y-offset and some scaling, have a look at the help of superCleanPlot
    mbTools.superCleanPlot(thedata.data['OE_LFP'], thedata.data['NPX'], structureLFP=['M1'], canauxNPX=[0], time=55) #canauxLFP=16, 
    picFN = os.path.sep.join([theExpe.rawDataPath,'A1-8978.svg'])
    plt.savefig(picFN, format="svg")

## Display

In [24]:
%gui qt
app = mkQApp()

#Create the main window that can contain several viewers
win = MainViewer(debug=True)

if 'LFP' in combined:
    source = InMemoryAnalogSignalSource(combined['LFP'], np.round(sample_rates['LFP']), t_start['LFP'], channel_names=channelLabels['LFP'])
    view1 = TraceViewer(source=source, name = 'LFP')

    #Parameters can be set in script
    view1.params['display_labels'] = True
    view1.params['scale_mode'] = 'same_for_all'
    view1.auto_scale()

    cmap = matplotlib.colormaps["hsv"]#Wistia"]
    nCh = len(view1.by_channel_params.children())
    for ch in range(nCh):
        #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
        #view1.by_channel_params[f'ch{ch}', 'offset'] = 0.1
        view1.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
        pass

    #create a time freq viewer conencted to the same source
    view2 = TimeFreqViewer(source=source, name='tfr')
    view2.params['show_axis'] = False
    view2.params['timefreq', 'deltafreq'] = 1
    #view2.by_channel_params['ch3', 'visible'] = False
    view2.auto_scale()

    win.add_view(view1)
    #win.add_view(view2)

if 'LFP_DS' in combined:

    if All_Spindle is not None:
        #Create one data source with 3 event channel
        all_events = []
        conditions = ['All','Good','Bad']
        for c,cond in enumerate(conditions):
            match cond:
                case 'All':
                    selection = "All_Spindle['toKeep'] | ~All_Spindle['toKeep']"
                case 'Good':
                    selection = "All_Spindle['toKeep']"
                case 'Bad':
                    selection = "~All_Spindle['toKeep']"
            ev_times = mbTools.convertTheoricIndex2realTime(All_Spindle.loc[pd.eval(selection),'peak time'].values, realFreq=sample_rates['LFP_DS'], offset=t_start['LFP_DS'])
            ev_labels = [f'spindle {i}'for i in All_Spindle[pd.eval(selection)].index]
            all_events.append({ 'time':ev_times, 'label':ev_labels, 'name': conditions[c] })
        source_ev = InMemoryEventSource(all_events=all_events)

        Spindle_peak = All_Spindle['peak time'].astype(int)
        Spindle_start = All_Spindle['start time'].astype(int)
        Spindle_end = All_Spindle['end time'].astype(int)

        #create 2 familly scatters from theses 2 indexes
        scatter_indexes = {0: Spindle_peak, 1: Spindle_start, 2: Spindle_end}
        #and asign them to some channels each
        scatter_channels = {0: [0], 1: [0], 2: [0]}
        source = AnalogSignalSourceWithScatter(combined['LFP_DS'], sample_rates['LFP_DS'], t_start['LFP_DS'], scatter_indexes, scatter_channels, channel_names=channelLabels['LFP_DS'])
        view_Events = EventList(source=source_ev, name='event')
        
    else:
        source = InMemoryAnalogSignalSource(combined['LFP_DS'], sample_rates['LFP_DS'], t_start['LFP_DS'], channel_names=channelLabels['LFP_DS'])
        view_Events = None
    view_DS = TraceViewer(source=source, name = 'LFP_DS')

    #Parameters can be set in script
    view_DS.params['display_labels'] = True
    view_DS.params['scale_mode'] = 'same_for_all'
    view_DS.auto_scale()

    cmap = matplotlib.colormaps["hsv"]#Wistia"]
    nCh = len(view_DS.by_channel_params.children())
    for ch in range(nCh):
        #view_DS.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
        #view_DS.by_channel_params[f'ch{ch}', 'offset'] = 0.1
        view_DS.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
        pass

    win.add_view(view_DS)
else:
    view_Events=None


if 'NPX' in combined:
    sig_source = SpikeInterfaceRecordingSource(recording=combined['NPX'])
    #view3 = TraceViewer.from_numpy(combined['NPX'], sample_rates['NPX'], t_start['NPX'], 'NPX', channel_names=channelLabels['NPX'])
    view3 = TraceViewer(source=sig_source, name='NPX')
    win.add_view(view3)

    #Parameters can be set in script
    view3.params['display_labels'] = True
    view3.params['scale_mode'] = 'same_for_all'
    view3.auto_scale()

    cmap = matplotlib.colormaps["hsv"]#Wistia"]
    nCh = len(view3.by_channel_params.children())
    for ch in range(nCh):
        #view3.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
        #view3.by_channel_params[f'ch{ch}', 'offset'] = 0.1
        view3.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
        pass

if view_Events is not None:
    win.add_view(view_Events)


#Run
win.show()
#app.exec()  #if commented, the app is shown and fonctionnal. Maybe detecting buttons. the Python icon doesn't close any better

debug True
QT_MODE PySide6
refresh duration for LFP_DS 9.393692016601562e-05 s
refresh duration for LFP_DS 8.082389831542969e-05 s
refresh duration for event 1.6689300537109375e-06 s


refresh duration for LFP_DS 0.0003540515899658203 s
